In [1]:
#!pip install pyvi
#!pip install stop-words
#!pip install stopwords
#!pip install googletrans
#!pip install google_trans_new
#!pip install deep_translator


In [2]:
from Trans_Tokenize import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.model_selection import train_test_split
import math
import unicodedata


In [3]:
raw_data_path='../dataset/job_data.csv'

raw_data=pd.read_csv(raw_data_path)
start = 500  # Vị trí bắt đầu (chỉ số hàng)
end = 520    # Vị trí kết thúc (không bao gồm hàng này)

raw_data = raw_data.iloc[start:end]

### Extract dataframe from few features

In [4]:
extracted_data=raw_data[['Company Name','Location','Experience','Job Level','Industry','Employment Type','Welfare','Job Description','Job Requirements','Follower','Company Address','Salary']]

### Drop some duplicate job application. Some company create the same after few day and change a bit 

In [5]:

extracted_data = extracted_data.drop_duplicates(subset=['Job Description','Job Level','Company Name','Location'], keep='first')


### Format Location

In [6]:

def remove_diacritics(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

In [7]:

extracted_data['Location'] = extracted_data['Location'].apply(lambda x: x.title())

extracted_data['Location'] = extracted_data['Location'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())
extracted_data['Location'] = extracted_data['Location'].apply(remove_diacritics)

filtered_location_data = extracted_data[~extracted_data['Location'].isin(['Not Specified', 'Not Found'])]



### Experience prepossessing

In [8]:
def extract_all_numbers(value):
    value = value.replace(',', '')
    value = value.replace('.', '')
    numbers = re.findall(r'\d+', value)
    numbers = [int(num) for num in numbers]
    
    if len(numbers) > 1 and (numbers[1] / numbers[0] > 2):
        return numbers[0]
    elif numbers:
        return sum(numbers) / len(numbers)  # Tính trung bình nếu có số
    return 0  
 

In [9]:
filtered_location_data.loc[:, 'Experience'] = filtered_location_data['Experience'].apply(extract_all_numbers)
filtered_experience_data = filtered_location_data

### Industry data

In [10]:

filtered_experience_data.loc[:,'Industry'] = filtered_experience_data['Industry'].apply(translate_column)

In [11]:
filtered_experience_data.loc[:,'Industry']=filtered_experience_data['Industry'].apply(process_tokenization_english)

### Employment Type

In [12]:
filtered_employee_data=filtered_experience_data.copy()
#filtered_employee_data['Employment Type'].value_counts()

In [16]:
filtered_employee_data.loc[:, 'Employment Type'] = filtered_employee_data['Employment Type'].replace({
    'Toàn thời gian': 'Official',
    'Nhân viên chính thức': 'Official',
    'Thời vụ/ Nghề tự do': 'Part-time',
    'Thực tập':'Internship'
})

### Job Level preprocessing

In [ ]:
filtered_job_level_data = filtered_employee_data.copy()
filtered_job_level_data.loc[:, 'Job Level'] = filtered_job_level_data['Job Level'].replace({
    'Nhân viên': 'Employee',
    'Trưởng nhóm / Giám sát': 'Leader',
    'Quản lý': 'Manager',
    'Giám đốc': 'Chief',
    'Sinh viên/ Thực tập sinh':'Intership',
})

### Welfare preprocessing

In [18]:
filtered_job_level_data.loc[:,'Welfare'] = filtered_job_level_data['Welfare'].apply(translate_column)

In [19]:
filtered_job_level_data.loc[:,'Welfare']=filtered_job_level_data['Welfare'].apply(process_tokenization_english)

In [20]:
filtered_welfare_data=filtered_job_level_data.copy()    

### Follower preprocessing

In [21]:

filtered_follower_data = filtered_welfare_data.copy()
filtered_follower_data['Follower'] = filtered_follower_data['Follower'].astype(str)

filtered_follower_data['Follower'] = filtered_follower_data['Follower'].replace('nan', '0').replace('Not Found', '0').replace('Not specified', '0')

filtered_follower_data['Follower'] = filtered_follower_data['Follower'].apply(lambda x: int(x.replace('.', '')))


### Job Requirements Preprocessing

In [22]:
filtered_requirement_data=filtered_follower_data.copy()
filtered_requirement_data['Job Requirements'] = filtered_requirement_data['Job Requirements'].replace(
    ['', 'Not Found', 'not found', 'Not specifield', 'Not Specifield'], 
    'No requirements needed')


In [23]:
filtered_requirement_data.loc[:,'Job Requirements'] = filtered_requirement_data['Job Requirements'].apply(translate_column)

In [24]:
filtered_requirement_data.loc[:,'Job Requirements']=filtered_requirement_data['Job Requirements'].apply(process_tokenization_english)

### Salary preprocessing

In [25]:
import re
import math

def extract_salary(value):
    # Loại bỏ dấu phẩy trong số (để xử lý 700,000 => 700000)
    value = value.replace(',', '')
    value = value.replace('.', '')
    
    # Tách các con số trong chuỗi
    numbers = re.findall(r'\d+', value)
    numbers = [int(num) for num in numbers]  # Chuyển các chuỗi số thành integer
    
    # Nếu có khoảng lương
    if len(numbers) > 1:
        # Nếu khoảng chênh lệch giữa hai giá trị quá lớn (ví dụ: 700 - 1,000 USD)
        if (numbers[1] - numbers[0]) / numbers[0] > 0.5:
            salary = numbers[0] + (numbers[1] - numbers[0]) / 4  # Đoán mức lương trung bình trong khoảng
        else:
            salary = sum(numbers) / len(numbers)  
    elif numbers:
        # Trường hợp chỉ có một số
        salary = sum(numbers) / len(numbers)
    else:
        return 'Cạnh tranh'
    
    # Kiểm tra các đơn vị tiền tệ và chuyển đổi
    if any(unit in value.lower() for unit in ['triệu', 'tr']):
        salary *= 1_000_000  
    elif any(unit in value.lower() for unit in ['usd', 'đô la', '$', 'đô']):
        salary *= 23_000  

    
    if any(unit in value.lower() for unit in ['năm', 'year']):
        salary /= 12  

    # Làm tròn kết quả về bội số gần nhất của 100,000
    return math.ceil(round(salary, -5))



In [26]:
filtered_salary_data = filtered_requirement_data.copy()

filtered_salary_data['Salary'] = filtered_salary_data['Salary'].replace(
    ['0','Thỏa thuận', 'Negotiable', 'Not Specified', 'Not Found', 'not found', 'Not specifield'], 
    np.nan)
filtered_salary_data['Salary'] = filtered_salary_data['Salary'].apply(extract_salary)

In [27]:
filtered_salary_data['Salary'].value_counts()

Salary
Cạnh tranh    6
12000000      2
12500000      1
22500000      1
21000000      1
41200000      1
16500000      1
15500000      1
9500000       1
600000000     1
20000000      1
7500000       1
Name: count, dtype: int64

In [28]:
processed_data=filtered_salary_data[['Location','Experience','Job Level','Industry','Employment Type','Welfare','Job Requirements','Follower','Salary']]

### Reduced Token


In [29]:
def remove_common_tokens(row):
    # Đảm bảo cả hai cột là chuỗi trước khi split
    job_requirements_text = ' '.join(row['Job Requirements']) if isinstance(row['Job Requirements'], list) else row['Job Requirements']
    welfare_text = ' '.join(row['Welfare']) if isinstance(row['Welfare'], list) else row['Welfare']
    
    # Tách các từ trong cả hai cột và chuyển thành set
    job_requirements_tokens = set(job_requirements_text.split())
    welfare_tokens = set(welfare_text.split())
    
    # Xóa các từ chung giữa Job Requirements và Welfare
    unique_tokens = job_requirements_tokens - welfare_tokens
    
    # Gộp các từ còn lại thành chuỗi và trả về
    return unique_tokens

In [30]:

processed_data.loc[:,'Job Requirements'] = processed_data.apply(remove_common_tokens, axis=1)


In [31]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 500 to 519
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Location          18 non-null     object
 1   Experience        18 non-null     object
 2   Job Level         18 non-null     object
 3   Industry          18 non-null     object
 4   Employment Type   18 non-null     object
 5   Welfare           18 non-null     object
 6   Job Requirements  18 non-null     object
 7   Follower          18 non-null     int64 
 8   Salary            18 non-null     object
dtypes: int64(1), object(8)
memory usage: 1.4+ KB


### Separate and write file

In [32]:
Defined_salary=processed_data[processed_data['Salary']!='Cạnh tranh']
Undefined_salary=processed_data[processed_data['Salary']=='Cạnh tranh']

In [33]:
Defined_salary.to_csv('../dataset/Defined_salary.csv', index=False,encoding='utf-8-sig')
Undefined_salary.to_csv('../dataset/Undefined_salary.csv', index=False,encoding='utf-8-sig')